In [ ]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from ultralytics import YOLO
import os
import shutil
import json
import matplotlib.pyplot as plt
import pandas as pd

# =========================
# 1. 모델 불러오기
# =========================
# model = YOLO("/content/drive/MyDrive/Posture/yolo11x-pose.pt")
model = YOLO("/content/drive/MyDrive/Posture/runs/yolo11x_pose_exp/weights/last.pt")

# =========================
# 2. 학습 설정
# =========================
project_dir = "/content/drive/MyDrive/Posture/runs"
exp_name = "yolo11x_pose_exp"
epochs = 400

results = model.train(
    data="/content/drive/MyDrive/Posture/coco8-pose.yaml",
    epochs=epochs,
    imgsz=512,
    save=True,
    resume=False,
    project=project_dir,
    name=exp_name,
    device=0,
    save_period=1
)

# =========================
# 3. best.pt / last.pt 복사
# =========================
def copy_with_correct_epoch(weights_dir, result_dir, target_name):
    target_path = os.path.join(weights_dir, target_name)
    if not os.path.exists(target_path):
        print(f"❌ {target_name} 파일이 존재하지 않습니다.")
        return

    results_json = os.path.join(result_dir, "results.json")
    if not os.path.exists(results_json):
        print("❌ results.json 파일이 존재하지 않습니다.")
        return

    with open(results_json, "r") as f:
        history = json.load(f)

    if target_name == "best.pt":
        best_epoch = history.get("best", None)
        if best_epoch is None:
            print("❌ best epoch 정보가 없습니다.")
            return
        print(f"✅ best.pt는 {best_epoch} epoch에서 저장됨")
        epoch_tag = best_epoch
    elif target_name == "last.pt":
        epoch_tag = len(history["metrics"]) - 1
        print(f"ℹ️ last.pt는 {epoch_tag} epoch에서 저장됨")
    else:
        print("❌ 지원되지 않는 파일 이름입니다.")
        return

    new_name = f"{os.path.splitext(target_name)[0]}_epoch{epoch_tag}.pt"
    new_path = os.path.join(weights_dir, new_name)
    shutil.copy(target_path, new_path)
    print(f"{target_name} → {new_name} 복사 완료 ✅")

# 경로
exp_dir = os.path.join(project_dir, exp_name)
weights_dir = os.path.join(exp_dir, "weights")

copy_with_correct_epoch(weights_dir, exp_dir, "best.pt")
copy_with_correct_epoch(weights_dir, exp_dir, "last.pt")

# =========================
# 4. 성능 지표 시각화 (results.csv)
# =========================
csv_path = os.path.join(exp_dir, "results.csv")
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    epochs = df['epoch']

    # 시각화: mAP50, box_loss, pose_loss
    plt.figure(figsize=(16, 5))

    plt.subplot(1, 3, 1)
    plt.plot(epochs, df['metrics/mAP50'], label='mAP@50', color='blue')
    plt.plot(epochs, df['metrics/mAP50-95'], label='mAP@50-95', color='green')
    plt.xlabel("Epoch")
    plt.ylabel("mAP")
    plt.title("mAP over Epochs")
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(epochs, df['loss/box'], label='Box Loss', color='red')
    plt.plot(epochs, df['loss/dfl'], label='DFL Loss', color='orange')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Box/DFL Loss")
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(epochs, df['loss/pose'], label='Pose Loss', color='purple')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Pose Loss")
    plt.legend()

    plt.tight_layout()
    plt.show()
else:
    print("❌ results.csv 파일이 존재하지 않습니다. 시각화를 건너뜁니다.")


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Posture/coco8-pose.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=400, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/Posture/yolo11x-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11x_pose_exp, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap

train: Scanning /content/datasets/coco-pose/labels/train2017.cache... 56599 images, 0 backgrounds, 0 corrupt: 100%|██████████| 56599/56599 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 958.1±822.7 MB/s, size: 168.6 KB)


val: Scanning /content/datasets/coco-pose/labels/val2017.cache... 2346 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2346/2346 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/Posture/runs/yolo11x_pose_exp/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 173 weight(decay=0.0), 183 weight(decay=0.0005), 182 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/Posture/runs/yolo11x_pose_exp
Starting training for 400 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     51/400      19.3G     0.8977      2.917     0.2991     0.7184      1.153         26        512: 100%|██████████| 3538/3538 [11:43<00:00,  5.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 74/74 [00:17<00:00,  4.27it/s]

                   all       2346       6352      0.877      0.865      0.937      0.756      0.855      0.818      0.869       0.62



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     52/400      19.3G     0.8955      2.927      0.299     0.7198      1.151         79        512:  91%|█████████ | 3224/3538 [10:42<01:01,  5.09it/s]